### Imports 

In [1]:
import numpy as np
import pygame as pg
import numpy as np

from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit import execute, BasicAer
from qiskit.compiler import transpile
from qiskit.extensions import RXGate, CnotGate, XGate
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit import Aer, execute
from qiskit.tools.visualization import plot_state_city
from qiskit.providers.aer import StatevectorSimulator

import matplotlib.pyplot as plt
import matplotlib.pyplot as plt
import thorpy
%matplotlib notebook

import cv2

pygame 1.9.6
Hello from the pygame community. https://www.pygame.org/contribute.html


### Qiskit Function 

In [2]:
#define colours
BLACK = (0,0,0)
WHITE = (255,255,255)
GREEN = (0,255,0)
RED = (255,0,0)

#Set width and height of the grid location
WIDTH = 20
HEIGHT = 20

#set margins between each cell
MARGIN = 5

#Create 2-D array as a list of lists
grid_size = 5

# Here we initialise our quantum and classical register
qr = QuantumRegister(2)
cr = ClassicalRegister(2)

# Construct the circuit
circ = QuantumCircuit(qr, cr)
circ.draw(output='mpl')
backend = Aer.get_backend('statevector_simulator')

In [3]:
def move_player(pos,d):
    x = pos[0]
    y = pos[1]
    #pos = None
    
    #for i in range(len(pos//2)):
    if len(pos) == 2:
        if d[0] == 'r':
            pos = [x + 1, y]
        if d[0] == 'l':
            pos = [x - 1, y]
        if d[0] == 'u':
            pos = [x, y - 1]
        if d[0] == 'd':
            pos = [x, y + 1]
        
    
    if len(pos) == 4:
        #print("HADAMARD!!!!!")
        if d[0] == 'r':
            pos = [x + 1, y,x - 1, y]
        if d[0] =='l':
            pos = [x - 1, y,x + 1, y]
        if d[0] =='u':
            pos = [x, y - 1,x, y+1]
        if d[0] =='d':
            pos = [x, y + 1,x, y-1]
        
    return pos

def get_dir(state_vector):
    #x = pos[0]
    #y = pos[1]
    #pos = None
    direction = None
    print(state_vector)
    if (state_vector == np.array([0,1,0,0])).all():
        direction = 'r'
    if (state_vector == np.array([0,0,1,0])).all():
        direction = 'l'
    if (state_vector == np.array([1,0,0,0])).all():
        direction = 'u'
    if (state_vector == np.array([0,0,0,1])).all():
        direction = 'd'
        
    #Account for Hadamar
    #test = state_vector
    #print("test:",np.array(test))
    test_vector = []
    
    for i in range(len(state_vector)):
        test_vector.append(float(state_vector[i]))
        
    test_vector = np.array(test_vector)
    
    #print("TEST")
    #print(test_vector)
    #print(len(state_vector))
    
    if np.allclose(test_vector, np.array([0.5,0.5,0.,0.])):
        direction = 'ur'
    if np.allclose(state_vector,np.array([0,0.5,0.5,0])):
        direction = 'ul'
    if np.allclose(state_vector,np.array([0,0,0.5,0.5])):
        direction = 'lu'  
    if np.allclose(state_vector,np.array([0.5,0,0,0.5])):
        direction = 'du'
        
    if np.allclose(state_vector,np.array([0.5,0,0.5,0])):
        direction = 'ru'
    
    print(direction)
    return direction

def mod_circ(circ,gate,qubit):
    
    if gate  == "H":
        circ.h(qubit)
        
    if gate  == "X":
        circ.x(qubit)
        
    if gate  == "cNOT":     
        if qubit == 0:
            circ.cx(0,1)
        else:
            circ.cx(1,0)
            
    if gate  == "ID":
        circ.iden(qubit)
        
    return circ

In [6]:
import pygame
from pygame.locals import *

pygame.init()

# first we define some constants
# doing this will reduce the amount of 'magic' numbers throughout the code
WINWIDTH = 1600
WINHEIGHT = 900
WINSIZE = (WINWIDTH, WINHEIGHT)

CELLWIDTH = 100
CELLHEIGHT = 100
CELLSIZE = (CELLWIDTH, CELLHEIGHT)

CELLMARGINX = 10 # number of pixels to the left and right of each cell
CELLMARGINY = 10# number of pixels to the top and bottom of each cell

SCREENPADX = 500 # number of pixels between the GRID and the left and right of the window
SCREENPADY = 50 # number of pixels between the GRID and the top and bottom of the window

WHITE = (255, 255, 255)
BLACK = (0, 0, 0)
RED = (255, 0, 0)
GREEN = (0, 255, 0)

DONE = False # is our program finished running?

# information about the two buttons (red and green)
#REDBUTTON = (0, 0, 120, 50)
q_s = 90
border = 10

QUBITBUTTON0 = (10,900-q_s - border, q_s,q_s)
QUBITBUTTON1 = (10,900-(q_s+border)*2,q_s, q_s)

g_s = 80

GATEBUTTON = (1510,border,g_s,g_s) # X
GATEBUTTON1 = (1510,border*2+g_s,g_s,g_s)# H
GATEBUTTON2 = (1510,border*3+g_s*2,g_s,g_s) # CNOT
GATEBUTTON3 = (1510,border*4+g_s*3,g_s,g_s)# UNITARY



# create the WINDOW and CLOCK
WINDOW = pygame.display.set_mode(WINSIZE)
pygame.display.set_caption('Qiskit')
CLOCK = pygame.time.Clock()

CURRENTCOLOR = RED # which color is active

# setting up the GRID
# cells can be accessed by GRID[row][col] ie. GRID[3][4] is the 3rd row and 4th column
# each cell contains [x, y, color]
# where x is the x position on the screen
#       y is the y position on the screen
#       color is the current color of the cell
GRID = []
for y in range(5):
    row = []
    for x in range(5):
        row.append([x * (CELLWIDTH + CELLMARGINX) + SCREENPADX, y * (CELLHEIGHT + CELLMARGINY) + SCREENPADY, WHITE])
    GRID.append(row)

# main loop
qubit = 0
gate = "ID"
pos = [2,2]
gate_selected = False
players = 1
num_H = 0

image_h = pg.image.load('./images/H.png').convert()
image_x = pg.image.load('./images/X.png').convert()
image_not = pg.image.load('./images/cNot.png').convert()
image_id = pg.image.load('./images/Id.png').convert()

while not DONE:
    isH = False
    # process all events
    for event in pygame.event.get():
        if event.type == QUIT: # did the user click the 'x' to close the window
            DONE = True

        if event.type == MOUSEBUTTONDOWN:
            # get the position of the mouse
            mpos_x, mpos_y = event.pos
            
            
            #------------------CHECK FOR QUBIT SELECTION------------------------------------------#
            # check if Qubit WAS CLICKED
            button_x_min, button_y_min, button_width, button_height = QUBITBUTTON0
            button_x_max, button_y_max = button_x_min + button_width, button_y_min + button_height
            if button_x_min <= mpos_x <= button_x_max and button_y_min <= mpos_y <= button_y_max:
                qubit = 0
                print("bit",0)
                
            button_x_min, button_y_min, button_width, button_height = QUBITBUTTON1
            button_x_max, button_y_max = button_x_min + button_width, button_y_min + button_height
            if button_x_min <= mpos_x <= button_x_max and button_y_min <= mpos_y <= button_y_max:
                qubit = 1
                print("bit",1)
            
            #-------------------CHECK FOR GATE SELECTION-----------------------------------------#
            # check if GATES were clicked
            button_x_min, button_y_min, button_width, button_height = GATEBUTTON
            button_x_max, button_y_max = button_x_min + button_width, button_y_min + button_height
            if button_x_min <= mpos_x <= button_x_max and button_y_min <= mpos_y <= button_y_max:
                #SELECT X GATE
                gate = 'X'
                circ.x(qubit)
                gate_selected  = True
                
            button_x_min, button_y_min, button_width, button_height = GATEBUTTON1
            button_x_max, button_y_max = button_x_min + button_width, button_y_min + button_height
            if button_x_min <= mpos_x <= button_x_max and button_y_min <= mpos_y <= button_y_max:
                #SELECT H GATE
                gate = 'H'
                circ.h(qubit)
                gate_selected  = True
                isH = True
                if isH and num_H < 1:
                #create copy
                    num_H += 1
                    players *= 2
                    pos.append(pos[players-1])
                    pos.append(pos[players])
                
            button_x_min, button_y_min, button_width, button_height = GATEBUTTON2
            button_x_max, button_y_max = button_x_min + button_width, button_y_min + button_height
            if button_x_min <= mpos_x <= button_x_max and button_y_min <= mpos_y <= button_y_max:
                #SELECT X GATE
                gate = 'CNOT'
                if qubit == 0:
                    circ.cx(0,1)
                else:
                    circ.cx(1,0)
                gate_selected  = True
                
            button_x_min, button_y_min, button_width, button_height = GATEBUTTON3
            button_x_max, button_y_max = button_x_min + button_width, button_y_min + button_height
            if button_x_min <= mpos_x <= button_x_max and button_y_min <= mpos_y <= button_y_max:
                #SELECT X GATE
                gate = 'ID' 
                circ.iden(qubit)
                gate_selected  = True
            
            built_circuit = circ.draw(output='mpl')
            built_circuit.savefig("test.png")
            #cv2.namedWindow("Circuit")
            #cv2.imshow("Input", "test.png")
            img = cv2.imread('test.png') # reads image 'opencv-logo.png' as grayscale
            cv2.namedWindow("Circuit")
            cv2.moveWindow("Circuit", 0,0)
            cv2.imshow("Circuit", img)
            
            #plt.imshow(img)
            #-------------------------Get output vector from inputs------------------------------#
            
            results = execute(circ,backend).result()
            state_vector = results.get_statevector(circ)
            direction = get_dir(np.abs(state_vector)**2)
            print(direction)
            
            
            #--------------------------Move Player---------------------------------------------#
            
            pos = move_player(pos,direction)
            
            # calculations for clicking cells

            mpos_x -= SCREENPADX # mouse position relative to the upper left cell
            mpos_y -= SCREENPADY # ^ same

            col = mpos_x // (CELLWIDTH + CELLMARGINX) # which cell is the mouse clicking
            row = mpos_y // (CELLHEIGHT + CELLMARGINY) # ^ same

            # make sure the user clicked on the GRID area
            if row >= 0 and col >= 0:
                try:
                    # calculate the boundaries of the cell
                    cell_x_min, cell_y_min =  col * (CELLHEIGHT + CELLMARGINY), row * (CELLWIDTH + CELLMARGINX)
                    cell_x_max = cell_x_min + CELLWIDTH
                    cell_y_max = cell_y_min + CELLHEIGHT
                    # now we will see if the user clicked the cell or the margin
                    if cell_x_min <= mpos_x <= cell_x_max and cell_y_min <= mpos_y <= cell_y_max:
                        GRID[row][col][2] = CURRENTCOLOR if event.button == 1 else WHITE
                    else:
                        # the user has clicked the margin, so we do nothing
                        pass
                except IndexError: # clicked outside of the GRID
                    pass           # we will do nothing

    # logic goes here


    # drawing
    WINDOW.fill(BLACK)
    
    # Make buttons
    #rect = image_h.get_rect()
    #rect.center = (200,300)
    
    pygame.draw.rect(WINDOW, RED, QUBITBUTTON0)
    pygame.draw.rect(WINDOW, RED, QUBITBUTTON1)
    h_gate = pygame.draw.rect(WINDOW, GREEN, GATEBUTTON)
    
    #img = pygame.Surface((100, 100))
    #image_file = pygame.image.load("./images/H.png")
    #img.blit(image_file, [0, 0])
    
    pygame.draw.rect(WINDOW, GREEN , GATEBUTTON1)
    pygame.draw.rect(WINDOW, GREEN , GATEBUTTON2)
    pygame.draw.rect(WINDOW, GREEN, GATEBUTTON3)
    pygame.Surface.blit(image_h, WINDOW, h_gate)
    
    for row in GRID:
        for x, y, color in row:
            pygame.draw.rect(WINDOW, color, (x, y, CELLWIDTH, CELLHEIGHT))
    #x,y,color = GRID[pos[0],pos[1]] 
    #x * (CELLWIDTH + CELLMARGINX) + SCREENPADX, y * (CELLHEIGHT + CELLMARGINY) + SCREENPADY
    f_red = (255, 0, 125)
    
    if(np.array(pos).any() < 0 or np.array(pos).any() >4):
        exit()
    
    if players == 1:
        pygame.draw.rect(WINDOW, RED, (pos[0]*(CELLWIDTH + CELLMARGINX) + SCREENPADX, 
                                       pos[1]*(CELLHEIGHT + CELLMARGINY) + SCREENPADY, CELLWIDTH, CELLHEIGHT))
    
    if players >= 2:
        
        pygame.draw.rect(WINDOW, f_red, (pos[0]*(CELLWIDTH + CELLMARGINX) + SCREENPADX, 
                                   pos[1]*(CELLHEIGHT + CELLMARGINY) + SCREENPADY, CELLWIDTH, CELLHEIGHT))
        
        pygame.draw.rect(WINDOW, f_red, (pos[2]*(CELLWIDTH + CELLMARGINX) + SCREENPADX, 
                                   pos[3]*(CELLHEIGHT + CELLMARGINY) + SCREENPADY, CELLWIDTH, CELLHEIGHT))
        
    pygame.draw.rect(WINDOW, GREEN, (4*(CELLWIDTH + CELLMARGINX) + SCREENPADX, 
                                   4*(CELLHEIGHT + CELLMARGINY) + SCREENPADY, CELLWIDTH, CELLHEIGHT))
    
    pygame.draw.rect(WINDOW, GREEN, (3*(CELLWIDTH + CELLMARGINX) + SCREENPADX, 
                                   4*(CELLHEIGHT + CELLMARGINY) + SCREENPADY, CELLWIDTH, CELLHEIGHT))
        
    #GRID[pos[0]][pos[1]] = RED
    pygame.display.flip()
    gate_selected = False
    CLOCK.tick(60)

pygame.quit()

[0.5 0.5 0.  0. ]
ur
ur
[0.5 0.5 0.  0. ]
ur
ur
bit 1
[0.5 0.5 0.  0. ]
ur
ur
[0.5 0.5 0.  0. ]
ur
ur
[0.  0.  0.5 0.5]
lu
lu
[0.5 0.5 0.  0. ]
ur
ur
[0.  0.  0.5 0.5]
lu
lu
[0.5 0.5 0.  0. ]
ur
ur
[0.  0.  0.5 0.5]
lu
lu
[0.5 0.5 0.  0. ]
ur
ur
[0.5 0.5 0.  0. ]
ur
ur
